In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0'}

In [3]:
# ссылки на страницы
num_of_page = 50
url_sp = []
for i in range(num_of_page):
    # получаем ссылки на каждую страницу
    url = 'https://nn.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p=' + str(i + 1) + '&region=4885&room3=1'
    
    #response = requests.get(url, headers=headers, verify=False)
    #print(response)
    
    url_sp.append(url)
# url_sp

In [4]:
# получаем список ссылок на каждую квартиру на каждой странице
list_hreff = []
for u in url_sp:
    response = requests.get(u, headers=headers, verify=False)
    response.encoding = 'utf8'
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    
    box = soup.find_all(class_='_93444fe79c--media--9P6wN')
    links = [link['href'] for link in box]
    for l in links:
        list_hreff.append(l)
# list_hreff

In [5]:
# переходим по ссылке на каждую квартиру и собираем инфу
result_df1 = pd.DataFrame()
for l in list_hreff:
    r = requests.get(l, headers=headers, verify=False)
    r.encoding = 'utf-8'
    content = r.text
    soup = BeautifulSoup(content, 'lxml')
    
    # заголовок
    title = soup.find('h1', class_='a10a3f92e9--title--vlZwT').get_text()
    
    # заголовок 2
    title2 = soup.find('title').get_text()
    
    # цена
    price = soup.find_all('span', class_='a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_9u--qr919 a10a3f92e9--fontWeight_bold--ePDnv a10a3f92e9--fontSize_28px--xlUV0 a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG')[-1].get_text().replace(u'\xa0', u' ')
    price = re.sub(r'[₽\s]', '', price).strip()
    
    # застройщик
    developer_el = soup.find('ul', class_='a10a3f92e9--container--z1RwI')
    if developer_el is not None: # проверяем, что элемент существует
        developer = developer_el.find('a', class_='a10a3f92e9--element--hhgQA').get_text() # получаем текст из элемента
    else:
        developer = 'No information' 
    
    # ссылка на квартиру
    link = soup.find("meta", attrs = {"property":"og:url"})['content']
    
    # описание от собственника
    descriptoin = soup.find('span', class_='a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_6u--A1GMI a10a3f92e9--fontWeight_normal--P9Ylg a10a3f92e9--fontSize_16px--RB9YW a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG a10a3f92e9--text_letterSpacing__0--mdnqq a10a3f92e9--text_whiteSpace__pre-wrap--scZwb').get_text()
    
    # площадь
    square = soup.find('span', class_='a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_6u--A1GMI a10a3f92e9--fontWeight_bold--ePDnv a10a3f92e9--fontSize_16px--RB9YW a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG').get_text().replace(u'\xa0', u' ')
    square = re.sub(r'м²', '', square).strip()
    square = re.sub(r',', '.', square) 
    
    # цена за 1 кв м
    price_one_metr = round(int(price)/float(square))
    
    # ЖК
    gk_element = soup.find('a', class_ = 'a10a3f92e9--link--A5SdC')
    if gk_element is not None: # проверяем, что элемент существует
        gk = gk_element.get_text() # получаем текст из элемента
    else:
        gk = 'No information'
    
    # адрес
    adr = soup.find_all("span", attrs = {"itemprop":"name"})[-1]['content']
    
    # этаж
    etage = soup.find('span', text = re.compile(' из '), attrs = {'class' : 'a10a3f92e9--color_black_100--kPHhJ a10a3f92e9--lineHeight_6u--A1GMI a10a3f92e9--fontWeight_bold--ePDnv a10a3f92e9--fontSize_16px--RB9YW a10a3f92e9--display_block--pDAEx a10a3f92e9--text--g9xAG'}).get_text()
    
    new_row = {'Цена': price, 'Цена за 1 кв.м': price_one_metr, 'Заголовок': title, 'Заголовок_2': title2,
                'Застройщик':developer, 'ЖК':gk, 'Ссылка': link, 'Описание':descriptoin, 'Площадь':square,
                'Адрес':adr, 'Этаж': etage}
    
    result_df1 = result_df1.append(new_row, ignore_index=True)
    #result_df1.to_excel('трешки.xlsx')